In [1]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path

In [2]:


def validate_files_exist(pattern, years):
    """Validate all required files exist"""
    missing_files = []
    for year in years:
        files = glob.glob(f'..\\data\\generated\\{pattern}_{year}.csv')
        if not files:
            missing_files.append(f"{pattern}_{year}.csv")
    if missing_files:
        raise FileNotFoundError(f"Missing files: {missing_files}")

def load_prediction_files(pattern, years=range(2025, 2040)):
    """Load and combine prediction CSVs"""
    validate_files_exist(pattern, years)
    dfs = []
    
    for year in years:
        file = glob.glob(f'..\\data\\generated\\{pattern}_{year}.csv')[0]
        df = pd.read_csv(file)
        
        # Validate required columns
        required_cols = ['Name', 'IDfg', 'Age', 'WAR']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in {file}: {missing_cols}")
            
        df['prediction_year'] = year
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Main data loading
try:
    sp_data = load_prediction_files('SP_Predictions')
    rp_data = load_prediction_files('RP_Predictions')
    batter_data = load_prediction_files('Batter_Predictions')
    salary_data = pd.read_csv('..\\data\\SPORTRAC_MLB_SALARY_DATA.csv')
    
    print(f"Loaded {len(sp_data)} SP predictions")
    print(f"Loaded {len(rp_data)} RP predictions")
    print(f"Loaded {len(batter_data)} batter predictions")
    print(f"Loaded {len(salary_data)} salary records")
    
except Exception as e:
    print(f"Error loading data: {str(e)}")

Loaded 3330 SP predictions
Loaded 6240 RP predictions
Loaded 6090 batter predictions
Loaded 3821 salary records


Group Positions, and merge data

In [3]:
# Add position grouping
sp_data['position_group'] = 'SP'
rp_data['position_group'] = 'RP'
batter_data['position_group'] = 'POS'

# Combine all prediction data
player_predictions = pd.concat([
    sp_data[['Name', 'IDfg', 'position_group', 'Age', 'prediction_year', 'WAR']],
    rp_data[['Name', 'IDfg', 'position_group', 'Age', 'prediction_year', 'WAR']],
    batter_data[['Name', 'IDfg', 'position_group', 'Age', 'prediction_year', 'WAR']]
], ignore_index=True)

# Print summary stats
print("\nPlayer Predictions Summary:")
print(player_predictions.groupby(['position_group', 'prediction_year'])['WAR'].agg(['count', 'mean']))

# Display first few rows of salary data to see structure
print("\nSalary Data Sample:")
print(salary_data.head())


Player Predictions Summary:
                                count      mean
position_group prediction_year                 
POS            2025               405  1.067714
               2026               405  0.582933
               2027               405  0.043274
               2028               405 -0.505815
               2029               405 -0.990998
               2030               405 -1.410560
               2031               405 -1.752162
               2032               405 -2.015884
               2033               405 -2.217236
               2034               405 -2.369951
               2035               405 -2.483591
               2036               405 -2.564493
               2037               405 -2.618696
               2038               405 -2.651803
               2039               405 -2.668152
RP             2025               416  0.706010
               2026               416  0.591346
               2027               416  0.523077
           

In [4]:
# Check predictions for duplicate names in same year
duplicate_check = player_predictions.groupby(['Name', 'prediction_year']).size().reset_index(name='count')
duplicates = duplicate_check[duplicate_check['count'] > 1]

print("Players with same name in same year:")
print(duplicates)

# Check salary data for duplicate names in same year
salary_duplicates = salary_data.groupby(['Player Name', 'Year']).size().reset_index(name='count')
salary_dupes = salary_duplicates[salary_duplicates['count'] > 1]

print("\nSalary records with same name in same year:")
print(salary_dupes)

Players with same name in same year:
              Name  prediction_year  count
9255   Logan Allen             2025      2
9256   Logan Allen             2026      2
9257   Logan Allen             2027      2
9258   Logan Allen             2028      2
9259   Logan Allen             2029      2
9260   Logan Allen             2030      2
9261   Logan Allen             2031      2
9262   Logan Allen             2032      2
9263   Logan Allen             2033      2
9264   Logan Allen             2034      2
9265   Logan Allen             2035      2
9266   Logan Allen             2036      2
9267   Logan Allen             2037      2
9268   Logan Allen             2038      2
9269   Logan Allen             2039      2
9450   Luis Garcia             2025      2
9451   Luis Garcia             2026      2
9452   Luis Garcia             2027      2
9453   Luis Garcia             2028      2
9454   Luis Garcia             2029      2
9455   Luis Garcia             2030      2
9456   Luis Garci

In [5]:
def identify_duplicates():
    # Combine prediction data
    all_predictions = pd.concat([
        sp_data[['Name', 'IDfg', 'position_group', 'WAR', 'prediction_year']],
        rp_data[['Name', 'IDfg', 'position_group', 'WAR', 'prediction_year']],
        batter_data[['Name', 'IDfg', 'position_group', 'WAR', 'prediction_year']]
    ])
    
    # Find duplicates
    duplicates = (
        all_predictions.groupby(['Name', 'prediction_year'])
        .filter(lambda x: len(x) > 1)
        .sort_values(['Name', 'prediction_year'])
    )
    
    # Save duplicates for review
    duplicates.to_csv('../data/generated/duplicate_players.csv', index=False)
    
    print("\nDuplicate Players Found:")
    for name in duplicates['Name'].unique():
        print(f"\nName: {name}")
        print(duplicates[duplicates['Name'] == name][['IDfg', 'position_group', 'prediction_year', 'WAR']])
    
    return duplicates

duplicates = identify_duplicates()
# After manual review, create mapping
name_to_idfg_mapping = {
    ('Luis Garcia', 'hou'): 6984,  # Example mapping
    ('Luis Garcia', 'was'): 20391,
    ('Will Smith', 'atl'): 6984,
    ('Will Smith', 'lad'): 19197
}


Duplicate Players Found:

Name: Logan Allen
       IDfg position_group  prediction_year  WAR
119   27589             SP             2025  1.6
215   18555             RP             2025  0.6
385   27589             SP             2026  0.6
620   18555             RP             2026  0.5
601   27589             SP             2027  0.3
1041  18555             RP             2027  0.4
823   27589             SP             2028 -0.1
1427  18555             RP             2028  0.4
1041  27589             SP             2029 -0.2
1835  18555             RP             2029  0.4
1253  27589             SP             2030 -0.3
2231  18555             RP             2030  0.4
1465  27589             SP             2031 -0.4
2649  18555             RP             2031  0.4
1675  27589             SP             2032 -0.5
3068  18555             RP             2032  0.4
1891  27589             SP             2033 -0.5
3491  18555             RP             2033  0.3
2102  27589             

In [6]:
def clean_salary_data(df):
    """Clean salary data removing special characters and handling percentages"""
    df = df.copy()
    
    # Remove non-player rows
    df = df[~df['Player Name'].str.contains('OPT-OUT|UFA', na=False)]
    
    # Convert Year to numeric, dropping non-year rows
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df = df.dropna(subset=['Year'])
    
    # Clean salary columns
    for col in ['Payroll', 'Luxury Tax']:
        # Handle percentage values
        mask_pct = df[col].str.contains('%', na=False)
        df.loc[mask_pct, col] = np.nan
        
        # Clean monetary values
        mask_money = ~mask_pct
        df.loc[mask_money, col] = (df.loc[mask_money, col]
            .replace(['-', ''], np.nan)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False))
        
        # Convert to float
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Select and rename columns
    clean_df = df[['Player Name', 'Year', 'Team', 'Payroll']].copy()
    clean_df = clean_df.rename(columns={'Payroll': 'Salary'})
    
    return clean_df

# Test cleaning
salary_data_clean = clean_salary_data(salary_data)
print("\nSample of cleaned salary data:")
print(salary_data_clean.head())

# Print summary stats
print("\nCleaning summary:")
print(f"Original rows: {len(salary_data)}")
print(f"Cleaned rows: {len(salary_data_clean)}")
print(f"Rows with valid salary: {salary_data_clean['Salary'].notna().sum()}")


Sample of cleaned salary data:
      Player Name    Year Team      Salary
0  Corbin Carroll  2023.0  ari   1625000.0
1  Corbin Carroll  2024.0  ari   3625000.0
2  Corbin Carroll  2025.0  ari   5625000.0
3  Corbin Carroll  2026.0  ari  10625000.0
4  Corbin Carroll  2027.0  ari  12625000.0

Cleaning summary:
Original rows: 3821
Cleaned rows: 3806
Rows with valid salary: 2043


In [7]:
def create_player_reference():
    """Create reference table of players with IDfg"""
    player_ref = pd.concat([
        sp_data[['Name', 'IDfg', 'position_group']],
        rp_data[['Name', 'IDfg', 'position_group']],
        batter_data[['Name', 'IDfg', 'position_group']]
    ]).drop_duplicates()
    
    return player_ref

# Create reference
player_ref = create_player_reference()

# Join with salary data
salary_with_id = salary_data.merge(
    player_ref[['Name', 'IDfg']], 
    left_on='Player Name',
    right_on='Name',
    how='left'
)

print("Players missing IDfg:", salary_with_id['IDfg'].isna().sum())
print("\nSample of unmatched players:")
print(salary_with_id[salary_with_id['IDfg'].isna()]['Player Name'].unique()[:10])

Players missing IDfg: 1205

Sample of unmatched players:
['Tim Tawa' 'Jordan Lawlar' 'Joe Elbis' 'Cristian Mena' 'Jorge Barrosa'
 'Yilber Diaz' 'Adrian Del Castillo' 'Blake Walston' 'Slade Cecconi'
 'Blaze Alexander']


In [8]:
def get_years_to_fa(status):
    """Calculate years until FA based on status"""
    if not status or pd.isna(status):
        return None
        
    status = status.upper()
    if 'ARB4' in status or 'ARB3' in status:
        return 1
    elif 'ARB2' in status:
        return 2
    elif 'ARB1' in status:
        return 3
    elif 'PRE-ARB' in status:
        return 4
    return None

def get_option_years(player_data):
    """Get all option years for player"""
    option_years = player_data[
        player_data['Status'].str.contains('OPT-OUT|PLAYER|CLUB|UFA', 
                                         na=False, 
                                         case=False)
    ].sort_values('Year')
    return option_years

def process_contract_status(salary_data):
    """Process all player contracts"""
    fa_years = {}
    
    valid_data = salary_data[
        (salary_data['IDfg'].notna()) & 
        (pd.to_numeric(salary_data['Year'], errors='coerce').notna())
    ].copy()
    
    valid_data['Year'] = pd.to_numeric(valid_data['Year'])
    
    for idfg, player_data in valid_data.groupby('IDfg'):
        latest = player_data.sort_values('Year', ascending=False).iloc[0]
        current_year = latest['Year']
        status = str(latest['Status']).upper().strip()
        
        # Handle Estimate (First year Pre-Arb)
        if status == 'Estimate':
            fa_years[idfg] = current_year + 6
            continue
        
        # Direct FA indicators
        if any(x in status for x in ['UFA', 'OPT-OUT', 'PLAYER']):
            fa_years[idfg] = current_year
            continue
        
        # Next year FA
        if any(x in status for x in ['CLUB', 'VESTING', 'ARB3', 'ARB 3', 'ARB4', 'ARB 4']):
            fa_years[idfg] = current_year + 1
            continue
        
        # ARB progression
        if any(x in status for x in ['ARB2', 'ARB 2']):
            fa_years[idfg] = current_year + 2
        elif any(x in status for x in ['ARB1', 'ARB 1']):
            fa_years[idfg] = current_year + 3
        elif 'PRE' in status and 'ARB' in status:
            fa_years[idfg] = current_year + 4
    
    return fa_years
# Run with debug output
fa_years = process_contract_status(salary_with_id)
salary_with_id['FA_Year'] = salary_with_id['IDfg'].map(fa_years)

# Print summary and check missing
print("\nFA Year Summary:")
print(f"Total players with FA year: {len(fa_years)}")
print(f"Players missing FA year: {salary_with_id['FA_Year'].isna().sum()}")
print("\nSample of players missing FA year:")
print(salary_with_id[salary_with_id['FA_Year'].isna()][['Player Name', 'Year', 'Status']].head())


FA Year Summary:
Total players with FA year: 676
Players missing FA year: 1215

Sample of players missing FA year:
      Player Name  Year    Status
72       Tim Tawa  2025  Estimate
75  Jordan Lawlar  2025  Estimate
76  Jordan Lawlar  2026   PRE-ARB
77  Jordan Lawlar  2027   PRE-ARB
78      Joe Elbis  2025  Estimate


In [9]:
WAR_VALUE_2025 = 8.5
INFLATION_RATE = 0.05
MIN_SALARY_2025 = 720000
ARB_PERCENTAGES = {
    'ARB1': 0.25,
    'ARB2': 0.33,
    'ARB3': 0.50,
    'ARB4': 0.70
}

In [10]:
def calculate_war_value(war, base_value=WAR_VALUE_2025):
    """Calculate non-linear WAR value using tiered system"""
    tier_multipliers = [
        (1, 1.0),  # 0-1 WAR
        (2, 1.5),  # 1-2 WAR
        (3, 2.0),  # 2-3 WAR
        (4, 2.5),  # 3-4 WAR
        (5, 3.0),  # 4-5 WAR
        (float('inf'), 4.0)  # 5+ WAR
    ]
    
    total_value = 0
    remaining_war = war
    
    for threshold, multiplier in tier_multipliers:
        war_in_tier = min(1, max(0, remaining_war))
        total_value += war_in_tier * base_value * multiplier
        remaining_war = max(0, remaining_war - 1)
        
        if remaining_war <= 0:
            break
            
    return total_value

In [11]:
def calculate_contract_value(war_value, status, year_offset):
    """Calculate contract value based on status"""
    min_salary = MIN_SALARY_2025 * (1 + INFLATION_RATE) ** year_offset
    
    if status == 'PRE-ARB':
        return min_salary
    
    for arb_year, percentage in ARB_PERCENTAGES.items():
        if arb_year in status:
            return max(min_salary, war_value * percentage)
    
    return war_value  # FA years

def calculate_player_values(predictions_df, salary_with_id):
    """Calculate player values before FA"""
    # Merge predictions with FA data
    value_df = predictions_df.merge(
        salary_with_id[['IDfg', 'FA_Year']].drop_duplicates(),
        on='IDfg',
        how='left'
    )
    
    # Setup calculations
    value_df['year_offset'] = value_df['prediction_year'] - 2025
    value_df['war_market_value'] = value_df.apply(
        lambda x: calculate_war_value(x['WAR']) * (1 + INFLATION_RATE) ** x['year_offset'],
        axis=1
    )
    
    # Get status and calculate contract values
    value_df['contract_value'] = value_df.apply(
        lambda x: calculate_contract_value(
            x['war_market_value'],
            x['Status'],
            x['year_offset']
        ) if x['prediction_year'] < x['FA_Year'] else x['war_market_value'],
        axis=1
    )
    
    value_df['surplus_value'] = value_df['war_market_value'] - value_df['contract_value']
    
    return value_df

In [12]:
# Combine all predictions into single DataFrame with required columns
all_predictions = pd.concat([
    sp_data[['Name', 'IDfg', 'position_group', 'prediction_year', 'WAR']],
    rp_data[['Name', 'IDfg', 'position_group', 'prediction_year', 'WAR']],
    batter_data[['Name', 'IDfg', 'position_group', 'prediction_year', 'WAR']]
], ignore_index=True)

In [13]:
# First, ensure both prediction_year and Year are same type (float)
# Clean salary data, removing invalid years
def clean_salary_df(df):
    """Clean salary dataframe removing rows with invalid years"""
    # Remove rows where Year contains text
    df = df[~df['Year'].str.contains('[a-zA-Z]', na=False, regex=True)]
    
    # Convert Year to numeric, dropping non-numeric years
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df = df.dropna(subset=['Year'])
    
    return df

# Clean salary data
salary_with_id = clean_salary_df(salary_with_id)

salary_with_id['Year'] = pd.to_numeric(salary_with_id['Year'])

C:\Users\User\AppData\Local\Temp\ipykernel_15160\2151017186.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = pd.to_numeric(df['Year'], errors='coerce')


In [14]:
# Modify the merge to include Team
value_df = all_predictions.merge(
    salary_with_id[['IDfg', 'Year', 'Status', 'FA_Year', 'Payroll', 'Team']], 
    left_on=['IDfg', 'prediction_year'],
    right_on=['IDfg', 'Year'],
    how='left'
)

In [15]:
# Add year offset and calculate WAR market value
value_df['year_offset'] = value_df['prediction_year'] - 2025
value_df['war_market_value'] = value_df.apply(
    lambda x: calculate_war_value(x['WAR']) * (1 + INFLATION_RATE) ** x['year_offset'],
    axis=1
)

In [16]:
def get_contract_value(row):
    """Get actual contract value or calculate theoretical value"""
    # If we have actual Payroll value, use it
    if pd.notna(row['Payroll']):
        return row['Payroll']
        
    # Otherwise calculate theoretical value
    if pd.isna(row['FA_Year']) or row['prediction_year'] >= row['FA_Year']:
        return row['war_market_value']  # Free agent value
        
    min_salary = MIN_SALARY_2025 * (1 + INFLATION_RATE) ** row['year_offset']
    
    # Pre-arb players
    if 'PRE-ARB' in str(row['Status']).upper():
        return min_salary
        
    # ARB players
    for arb_year, percentage in ARB_PERCENTAGES.items():
        if arb_year in str(row['Status']).upper():
            return max(min_salary, row['war_market_value'] * percentage)
            
    return row['war_market_value']

value_df['contract_value'] = value_df.apply(get_contract_value, axis=1)

In [17]:
value_df_clean = value_df.dropna(subset=['Payroll'])
value_df_clean['Payroll'] = pd.to_numeric(
    value_df_clean['Payroll'].str.replace('$', '').str.replace(',', ''), 
    errors='coerce'
) / 1000000  # Convert to millions

# Debug prints
print(f"Total rows before filtering: {len(value_df)}")
print(f"Rows with valid payroll data: {len(value_df_clean)}")
print("\nData types:")
print(value_df_clean[['war_market_value', 'Payroll']].dtypes)

# Calculate surplus value (both values now in millions)
value_df_clean['surplus_value'] = value_df_clean['war_market_value'] - value_df_clean['Payroll']

# Group by player to get total value
player_values = value_df_clean.groupby(['IDfg', 'Team'])['surplus_value'].sum().reset_index()

# Print results
print(f"\nNumber of players with calculated values: {len(player_values)}")
print("\nSample of player values:")
print(player_values.merge(
    player_ref[['Name', 'IDfg']], 
    on='IDfg',
    how='left'
).head())

Total rows before filtering: 15667
Rows with valid payroll data: 1140

Data types:
war_market_value    float64
Payroll             float64
dtype: object

Number of players with calculated values: 684

Sample of player values:
   IDfg Team  surplus_value               Name
0  2036  lad       3.600000    Clayton Kershaw
1  3237  ath       4.150000     T.J. McFarland
2  4301  bos       1.150000      Justin Wilson
3  4949  nyy     -81.000000  Giancarlo Stanton
4  5361  lad     -20.816971    Freddie Freeman


C:\Users\User\AppData\Local\Temp\ipykernel_15160\4027554796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_df_clean['Payroll'] = pd.to_numeric(
C:\Users\User\AppData\Local\Temp\ipykernel_15160\4027554796.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_df_clean['surplus_value'] = value_df_clean['war_market_value'] - value_df_clean['Payroll']


In [ ]:
# Create output directory if it doesn't exist
output_dir = '../data/generated/value_by_year'
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Prepare yearly data
for year in range(2025, 2040):
    # Filter data for current year
    year_data = value_df_clean[value_df_clean['prediction_year'] == year].copy()
    
    # Select and rename columns
    output_data = year_data[[
        'IDfg', 
        'Name', 
        'Team', 
        'contract_value',
        'war_market_value',
        'surplus_value'
    ]].copy()
    
    # Round numeric columns to 2 decimals
    numeric_cols = ['contract_value', 'war_market_value', 'surplus_value']
    output_data[numeric_cols] = output_data[numeric_cols].round(2)
    
    # Sort by surplus value
    output_data = output_data.sort_values('surplus_value', ascending=False)
    
    # Save to CSV
    output_path = f'{output_dir}/player_values_{year}.csv'
    output_data.to_csv(output_path, index=False)
    
    print(f"Saved {len(output_data)} player values for {year}")

print("\nCompleted saving all yearly value files")


Results saved to: ../data/generated/team_surplus_values.csv

Top 5 teams by surplus value:
   Team  surplus_value  war_market_value  Payroll
12   kc         724.28           1270.67   546.40
2   atl         584.75           1227.82   643.07
7   cin         393.52            554.30   160.78
3   bal         384.52            518.93   134.41
23  sea         354.31            796.01   441.71
